## preprocessing - (to3tuple)

In [3]:
import pandas as pd
import numpy as np
df = pd.read_csv('Reviews_sa.csv')
df['score_with_sa'] = df['Score']-df['sa_compound_score']
df

,Unnamed: 0,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,sa_compound_score,score_with_sa
0,0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,0.9441,4.0559
1,1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,-0.5664,1.5664
2,2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,0.8265,3.1735
3,3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,0.0000,2.0000
4,4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,0.9468,4.0532
...,...,...,...,...,...,...,...,...,...,...,...,...,...
568449,568449,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,5,1299628800,Will not do without,Great for sesame chicken..this is a good if no...,0.8589,4.1411
568450,568450,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,2,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...,-0.4848,2.4848
568451,568451,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,5,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o...",0.4352,4.5648
568452,568452,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,5,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...,0.9717,4.0283


In [4]:
## generate ratings
users = df['UserId'].unique()
items = df['ProductId'].unique()
users_dict = {users[i]:i for i in range(len(users))}
items_dict = {items[i]:i for i in range(len(items))}
intUserId = df.apply(lambda x: users_dict[x['UserId']], axis=1)
intItemId = df.apply(lambda x: items_dict[x['ProductId']], axis=1)
ratings = pd.concat([intUserId,intItemId,df['Score']], axis=1)
ratings.to_csv('Ratings.csv', index=False, header=False)

In [5]:
df_vip = df[df.groupby('UserId').UserId.transform(len) > 1]

In [10]:
#remove users appears once and generate ratings
users = df_vip['UserId'].unique()
items = df_vip['ProductId'].unique()
users_dict = {users[i]:i for i in range(len(users))}
items_dict = {items[i]:i for i in range(len(items))}
intUserId = df_vip.apply(lambda x: users_dict[x['UserId']], axis=1)
intItemId = df_vip.apply(lambda x: items_dict[x['ProductId']], axis=1)
ratings = pd.concat([intUserId,intItemId,df_vip['Score']], axis=1)
ratings.to_csv('Ratings_vip.csv', index=False, header=False)
print(df.size, df_vip.size)

7389902 5109819


In [21]:
# prepare for association rule
bucket_dict = {}
for index, row in df_vip.iterrows():
    bucket_dict.setdefault(row["UserId"], []).append(row["ProductId"])

In [25]:
bucket_dict

{'ABXLMWJIXXAIN': ['B000LQOCH0', 'B000CQ26E0', 'B004DMGQKE'],
 'A395BORC6FGVXV': ['B000UA0QIQ', 'B002Y7526Y', 'B000U9WZ54'],
 'A1UQRSCLF8GW1T': ['B006K2ZZ7K', 'B003XDH6M6'],
 'A1SP2KVKFXXRU1': ['B006K2ZZ7K', 'B0058AVD44'],
 'A1MZYO9TZK0BBI': ['B000E7L2R4', 'B000E7QYNG'],
 'A21BT40VZCCYT4': ['B00171APVA', 'B0006349W6', 'B0006349WQ', 'B0006349WG'],
 'A3HDKO7OW0QNK4': ['B0001PB9FE', 'B0001PB9FY', 'B0001PB9EK', 'B0001PB9EA'],
 'A2725IB4YY9JEB': ['B0009XLVG0',
  'B001L1KH6Y',
  'B003THML30',
  'B004YWHTBS',
  'B003WLC4VC',
  'B003THQKXM',
  'B003NZHOQ2',
  'B005LXGGH2',
  'B000G82L5I',
  'B000FBQ5EY',
  'B003THTKZC',
  'B003UE2X2G'],
 'A327PCT23YH90': ['B0009XLVG0', 'B0009XJQ1W'],
 'A18ECVX2RJ7HUE': ['B001GVISJM',
  'B004ZIER34',
  'B007JFMH96',
  'B0041NYV8E',
  'B007I7Z3Z0',
  'B004MO6NI8',
  'B005XQYTXA',
  'B000HDL1P8',
  'B000LQORIY',
  'B002I0T8VC',
  'B000LKZ86K',
  'B0015V8EOW',
  'B001EO605K',
  'B002IEVJRY',
  'B004SJ8IUI',
  'B007JFMIWW',
  'B002TMV3CG',
  'B003XDH6M6',
  'B006BX

In [31]:
import csv
import math
import random
bucketTrain = open("bucket_train.csv","w")
bucketTest = open("bucket_test.csv","w")
for key, val in bucket_dict.items():
    if random.random()<0.1:
        if len(val)>20:
            bucketTest.write('\n'+','.join(val[:20]))
        if len(val)<20:
            nans = ['NaN']*(20-len(val))
            bucketTest.write('\n'+','.join(val)+','+','.join(nans))
    else:
        if len(val)>20:
            bucketTrain.write('\n'+','.join(val[:20]))
        if len(val)<20:
            nans = ['NaN']*(20-len(val))
            bucketTrain.write('\n'+','.join(val)+','+','.join(nans))

In [ ]:
# add in sentiment analysis
users = df_vip['UserId'].unique()
items = df_vip['ProductId'].unique()
users_dict = {users[i]:i for i in range(len(users))}
items_dict = {items[i]:i for i in range(len(items))}
intUserId = df_vip.apply(lambda x: users_dict[x['UserId']], axis=1)
intItemId = df_vip.apply(lambda x: items_dict[x['ProductId']], axis=1)
ratings = pd.concat([intUserId,intItemId,df_vip['score_with_sa']], axis=1)
ratings.to_csv('Ratings_vip_sa.csv', index=False, header=False)

In [ ]:
#moderate user scores based on user input and group input and generate ratings
df_vip['AvgUserScore'] = df_vip['Score'].groupby(df_vip['UserId']).transform('mean')
df_vip['AvgProductScore'] = df_vip['Score'].groupby(df_vip['ProductId']).transform('mean')
df_vip['ModeratedScore'] = 0.75*df_vip['AvgUserScore'] + 0.25*df_vip['AvgProductScore']
users = df_vip['UserId'].unique()
items = df_vip['ProductId'].unique()
users_dict = {users[i]:i for i in range(len(users))}
items_dict = {items[i]:i for i in range(len(items))}
intUserId = df_vip.apply(lambda x: users_dict[x['UserId']], axis=1)
intItemId = df_vip.apply(lambda x: items_dict[x['ProductId']], axis=1)
ratings = pd.concat([intUserId,intItemId,df_vip['ModeratedScore']], axis=1)
ratings.to_csv('Ratings_vip_moderateS.csv', index=False, header=False)